In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# fetching dataset 
ss=10
np.random.seed(2)
x0=np.random.uniform(-0.1,0.1,(ss,1))
x1=np.random.uniform(0.9,1.1,(ss,1))

y0=-1*np.ones((ss,1))
y1=np.ones((ss,1))

d00=np.concatenate((x0,x0,y0),axis=1)
d01=np.concatenate((x0,x1,y1),axis=1)
d10=np.concatenate((x1,x0,y1),axis=1)
d11=np.concatenate((x1,x1,y0),axis=1)
d=np.concatenate((d00,d01,d10,d11),axis=0)
x_xor=np.zeros((d.shape[0],2))
y_xor=np.zeros((d.shape[0],1))

x_xor[:,:]=d[:,0:2]
y_xor[:,0]=d[:,2]

N=len(y_xor)

x_test=np.matrix('0.0,0.0;0.0,1.0;1.0,0.0;1.0,1.0')
y_test=np.matrix('-1.0;1.0;1.0;-1.0')




def fit(x,y,N,k_type,sigma=0.6,c=1,p=4):
    global w,b,sv_al,sv_y,sv_x,kr_type  
    
    kr_type=k_type
    
    from cvxopt import matrix,solvers

    K=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            kernel= {'linear':np.dot(x[i], x[j]),'polynomial':(c + np.dot(x[i], x[j])) ** p,
                 'rbf':np.exp(-np.linalg.norm(x[i]-x[j])**2 /(2*(sigma ** 2)))}
            K[i,j]=kernel[kr_type]
    P=matrix(np.outer(y,y)*K)
    q=matrix(-1*np.ones((N)))
    A=matrix(np.transpose(y))
    b=matrix(np.zeros((1,1)))
    G=matrix(-1*np.identity(N))
    h=matrix(np.zeros((N,1)))
    sol = solvers.qp(P,q,G,h,A,b)
    al=np.ravel(sol['x'])    

    # Support vectors have non zero lagrange multipliers
    sv = al > 1e-5
    ind = np.arange(len(al))[sv]
    sv_al = al[sv]
    sv_x = x[sv]
    sv_y = y[sv]
    
    
    # Intercept
    b = 1e-10
    for n in range(len(sv_al)):
        b += sv_y[n]
        b -= np.sum(sv_al*sv_y*K[ind[n],sv])
    b /= len(sv_al)
    
    # Weight vector
    if kr_type == 'linear':
        w = np.zeros(2)
        for n in range(len(sv_al)):
            w += sv_al[n] * sv_y[n] * sv_x[n]
    else:
        w = None
    
    
def predict(X,sigma=0.6,c=1,p=4):

    if w is not None:
        return np.sign(np.dot(X,w) + b)
    
    else:
        y_predict = np.zeros(len(X))
        for i in range(len(X)):
            sc = 0
            for n in range(len(sv_al)):
                kernel= {'polynomial':(c + np.dot(X[i], sv_x[n])) ** p,
                         'rbf':np.exp(-np.linalg.norm(X[i]-sv_x[n])**2 /(2*(sigma ** 2)))}
                sc += sv_al[n] * sv_y[n] *kernel[kr_type]
            y_predict[i] = sc
        return np.sign(y_predict + b)


def test(x_Q):
    pred=[]
    for i in range(len(x_Q)):
        c=predict(x_Q[i,:])
        pred.append(c)
    return np.array(pred)

def accuracy(pred,y):
    a=[]
    print(len(y))
    for i in range(len(y)):
        if pred[i]==y[i]:
            a.append(1)
        else:
            a.append(0)
                
    acc=100*np.mean(a)
    return acc





fit(x_xor,y_xor,N,'linear')
pred_lin=test(x_test)
acc=accuracy(pred_lin,y_test)
print('linear prediction==',pred_lin)
print('linear prediction accuracy==',acc)



fit(x_xor,y_xor,N,'polynomial')
pred_poly=test(x_test)
acc=accuracy(pred_poly,y_test)
print('polynomial prediction==',pred_poly)
print('polynomial prediction accuracy==',acc)



fit(x_xor,y_xor,N,'rbf')
pred_rbf=test(x_test)
acc=accuracy(pred_rbf,y_test)
print('rbf prediction==',pred_rbf)
print('rbf prediction accuracy==',acc)

     pcost       dcost       gap    pres   dres
 0: -4.0000e+01 -8.0000e+01  4e+01  7e-15  2e+00
 1: -9.9400e+01 -1.0039e+02  1e+00  4e-15  1e+00
 2: -1.0037e+04 -1.0038e+04  1e+00  1e-12  1e+00
 3: -9.9387e+07 -9.9387e+07  1e+02  1e-07  1e+00
 4: -4.0641e+10 -4.0641e+10  4e+04  2e-06  1e+00
Terminated (singular KKT matrix).
4
linear prediction== [[[1.]]

 [[1.]]

 [[1.]]

 [[1.]]]
linear prediction accuracy== 50.0
     pcost       dcost       gap    pres   dres
 0: -1.6711e+00 -3.2465e+00  7e+01  8e+00  1e+00
 1: -3.7783e-01 -2.0528e+00  5e+00  4e-01  6e-02
 2: -4.2927e-01 -1.0633e+00  6e-01  1e-16  2e-15
 3: -5.6817e-01 -6.5926e-01  9e-02  2e-16  1e-15
 4: -5.9833e-01 -6.7086e-01  7e-02  1e-16  1e-15
 5: -6.4874e-01 -6.5265e-01  4e-03  1e-16  1e-15
 6: -6.5173e-01 -6.5177e-01  4e-05  2e-16  2e-15
 7: -6.5176e-01 -6.5176e-01  4e-07  4e-16  2e-15
Optimal solution found.
4
polynomial prediction== [[ 1.]
 [ 1.]
 [ 1.]
 [-1.]]
polynomial prediction accuracy== 75.0
     pcost       dcost  

from the above output we are getting an accuracy of
linear prediction accuracy = 50%
polynomial prediction accuracy = 75%
rbf prediction accuracy = 100%